In [1]:
import numpy as np
import cv2
import time
import os
import os.path
import glob
from multiprocessing import Pool, Process
import math
import gc
import datetime

ver = cv2.__version__
print('OpenCV Version: ' + ver)

OpenCV Version: 3.3.1


In [2]:
# %qtconsole

In [4]:
# find list of all videos
vids = glob.glob('/media/gravishlab/Seagate Expansion Drive/AntTrack/*0000.avi')
print('Total Number of Videos: ',len(vids))
vids = sorted(vids)

# dest folder
dfolder = '/media/gravishlab/Seagate Expansion Drive/AntTrack/AntTunnel/5mm_clear' # if running as part of trigger cams
# dfolder = '/media/gravishlab/Seagate Expansion Drive/AntTrack/0mm_clear'
if not os.path.exists(dfolder):
    os.makedirs(dfolder)
                           

Total Number of Videos:  709


In [10]:
# DEFINE FUNCTION TO TEST IF ANTS IN VIDEOS USING BACKGROUND SUBTRACTION

def loadvideo(vname, sep = 20):
    
    # capture video, and find number of frames
    cap = cv2.VideoCapture(vname)
    vFr = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    print('---N_Frames', vFr)
    
    nFr = math.floor(vFr/sep)+1
    frames = np.zeros((nFr, h, w), np.uint8)
    
    for kk,fr in enumerate(list(range(vFr))[::sep]):
        cap.set(1,fr) # define frame (must be between 0 and 1)
        tru, ret = cap.read(1)
#         tru, ret = cap.read(fr)

        # check if video frames are being loaded
        if not tru:
            print('Codec issue: cannot load frames.')
            break
            
        frames[kk,:,:] = ret[:,:,0]

#         if ((fr % 100) == 0):
#             print('---',fr)
            
#     print('---Video loaded')
    cap.release()
    gc.collect()
    return frames

    
def computebkgd(frames, bkgd_sep = 1):
    
    bkgd = np.float32(np.median(frames[0::bkgd_sep,:,:], axis = 0))
    bkgd = bkgd + np.float32(1E-6)
    return bkgd


def find_threshold(frames):
    [nfr,h,w] = frames.shape
    
    # get rid of salt and pepper noise
#     temp = [cv2.medianBlur(fr,3) for fr in frames]
#     median_blur = np.asarray(temp)
    
    # find how many points are above a threshold
#     thresholded=median_blur[0+50-1:(h-50-1), 0+50-1:(w-120-1)]<.8 
    thresholded=frames[:,0+50-1:(h-50-1), 0+50-1:(w-120-1)]<.8 
    return thresholded
    


def deleteantvids(vname):
    temp = vname.split('/')
    vbase = temp[-1].split('.')[0][0:-7]
    dbase = temp[-1].split('.')[0].split('_')[0]
    tbase = temp[-1].split('.')[0].split('_')[1]
    
    # delete videos
    os.remove(vname)
    if os.path.exists(vbase + '12-0000.avi'):
        os.remove(vbase + '12-0000.avi')
    if os.path.exists(vbase + '35-0000.avi'):
        os.remove(vbase + '35-0000.avi')
    if os.path.exists(vbase + '36-0000.avi'):
        os.remove(vbase + '36-0000.avi')

    # webcam pic
    vtime1 = (datetime.datetime.strptime(tbase,'%H%M%S') - datetime.timedelta(seconds=3)).strftime('%H%M%S')
    vtime2 = (datetime.datetime.strptime(tbase,'%H%M%S') - datetime.timedelta(seconds=4)).strftime('%H%M%S')
    vname1 = dbase + '_' + vtime1 + '.jpg'
    vname2 = dbase + '_' + vtime2 + '.jpg'
    if os.path.exists(('/').join(temp[:-1]) + '/' + vname1):
        os.remove(('/').join(temp[:-1]) + '/' + vname1)
    if os.path.exists(('/').join(temp[:-1]) + '/' + vname2):
        os.remove(('/').join(temp[:-1]) + '/' + vname2)

#     print('---**Videos deleted**')
    return;


def movevids(vname):
    temp = vname.split('/')
    dbase = temp[-1].split('.')[0].split('_')[0]
    tbase = temp[-1].split('.')[0].split('_')[1]
    
    # move videos
    os.rename(vname, dfolder + '/' + temp[-1])
#     if os.path.exists(vname[0:len(vname)-11] + '12-0000.avi'):
#         os.remove(vname[0:len(vname)-11] + '12-0000.avi')
#     if os.path.exists(vname[0:len(vname)-11] + '35-0000.avi'):
#         os.remove(vname[0:len(vname)-11] + '35-0000.avi')
#     if os.path.exists(vname[0:len(vname)-11] + '36-0000.avi'):
#         os.remove(vname[0:len(vname)-11] + '36-0000.avi')

    # webcam pic
    vtime1 = (datetime.datetime.strptime(tbase,'%H%M%S') - datetime.timedelta(seconds=3)).strftime('%H%M%S')
    vtime2 = (datetime.datetime.strptime(tbase,'%H%M%S') - datetime.timedelta(seconds=4)).strftime('%H%M%S')
    vname1 = dbase + '_' + vtime1 + '.jpg'
    vname2 = dbase + '_' + vtime2 + '.jpg'

    if os.path.exists(('/').join(temp[:-1]) + '/' + vname1):
#         print(dfolder + '/' + vname1)
        os.rename(('/').join(temp[:-1]) + '/' + vname1, dfolder + '/' + vname1)
    if os.path.exists(('/').join(temp[:-1]) + '/' + vname2):
        os.rename(('/').join(temp[:-1]) + '/' + vname2, dfolder + '/' + vname2)



def findants(vname):
    t = time.time()
#     print(vname,'asge')
    
    frames = loadvideo(vname)
#     print('frames loaded ', str(time.time()-t))
    bkgd = computebkgd(frames, bkgd_sep = 10)
#     print('bkgd subtracted ', str(time.time()-t))
    norm_bkgd = np.mean(bkgd[:])
    bkgd_div = (frames/norm_bkgd) / (bkgd/norm_bkgd)
#     print('bkgd divided ', str(time.time()-t))
    [h,w] = bkgd.shape
    thresholded = find_threshold(bkgd_div)
#     print('thresholded ', str(time.time()-t))
    totals = [th.sum() for th in thresholded]
#     print('ant pix calculated ', str(time.time()-t))
    
    ant_frames = np.asarray(totals) > 500        # when more than __ pixels are above threshold
    ant_present[vids.index(vname),0]=ant_frames.sum() # how many frames with ant in them
    print('---# frames with ant mvmt: ', ant_frames.sum(),'/',str(math.floor(717/20)+1))


    if ant_frames.sum() < 4:
        deleteantvids(vname)
    else:
        movevids(vname)

#     del frames, bkgd, norm_bkgd, bkgd_div, thresholded, totals, ant_frames, ant_present
    print(vname, ' -- %.2f s' % (time.time()-t))
    return



In [11]:
# DEFINE FUNCTION TO TEST IF ANTS IN VIDEOS USING ONLY 2 FRAMES

def loadendframes(vname, sep = 50):
    
    # capture video, and find number of frames
    cap = cv2.VideoCapture(vname)
    vFr = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
#     print('---N_Frames', vFr)
    
    frames = np.zeros((2, h, w), np.uint8)
    
    for kk,fr in enumerate([sep, vFr-sep]):
        cap.set(1,fr) # define frame (must be between 0 and 1)
        tru, ret = cap.read(1)

        # check if video frames are being loaded
        if not tru:
            print('Codec issue: cannot load frames.')
            break
            
        frames[kk,:,:] = ret[:,:,0]

#         cv2.imshow('raw',ret)
#         cv2.waitKey(250)
#         cv2.destroyAllWindows()
            
    cap.release()
    return frames


def subtractframes(frames):
    fdiff = np.diff(frames.astype(np.int16), n=1, axis = 0)
    bigdiff = (fdiff < -10) | (fdiff > 10)
    bigdiff = np.swapaxes(np.swapaxes(bigdiff,0,1),1,2) # make 3rd axis singular so works for imshow
    ndiff = bigdiff.sum()
#     cv2.imshow('raw', bigdiff.astype(np.uint8)*255)
#     cv2.waitKey(1000)
#     cv2.destroyAllWindows()
    
    return ndiff


def deleteantvids(vname):
    temp = vname.split('/')
    vbase = temp[-1].split('.')[0][0:-7]
    dbase = temp[-1].split('.')[0].split('_')[0]
    tbase = temp[-1].split('.')[0].split('_')[1]
    
    # delete videos
    os.remove(vname)
    if os.path.exists(vbase + '12-0000.avi'):
        os.remove(vbase + '12-0000.avi')
    if os.path.exists(vbase + '35-0000.avi'):
        os.remove(vbase + '35-0000.avi')
    if os.path.exists(vbase + '36-0000.avi'):
        os.remove(vbase + '36-0000.avi')

    # webcam pic
    vtime1 = (datetime.datetime.strptime(tbase,'%H%M%S') - datetime.timedelta(seconds=3)).strftime('%H%M%S')
    vtime2 = (datetime.datetime.strptime(tbase,'%H%M%S') - datetime.timedelta(seconds=4)).strftime('%H%M%S')
    vname1 = dbase + '_' + vtime1 + '.jpg'
    vname2 = dbase + '_' + vtime2 + '.jpg'
    if os.path.exists(('/').join(temp[:-1]) + '/' + vname1):
        os.remove(('/').join(temp[:-1]) + '/' + vname1)
    if os.path.exists(('/').join(temp[:-1]) + '/' + vname2):
        os.remove(('/').join(temp[:-1]) + '/' + vname2)

#     print('---**Videos deleted**')
    return;


def movevids(vname):
    temp = vname.split('/')
    dbase = temp[-1].split('.')[0].split('_')[0]
    tbase = temp[-1].split('.')[0].split('_')[1]
    
    # move videos
    os.rename(vname, dfolder + '/' + temp[-1])
#     if os.path.exists(vname[0:len(vname)-11] + '12-0000.avi'):
#         os.remove(vname[0:len(vname)-11] + '12-0000.avi')
#     if os.path.exists(vname[0:len(vname)-11] + '35-0000.avi'):
#         os.remove(vname[0:len(vname)-11] + '35-0000.avi')
#     if os.path.exists(vname[0:len(vname)-11] + '36-0000.avi'):
#         os.remove(vname[0:len(vname)-11] + '36-0000.avi')

    # webcam pic
    vtime1 = (datetime.datetime.strptime(tbase,'%H%M%S') - datetime.timedelta(seconds=3)).strftime('%H%M%S')
    vtime2 = (datetime.datetime.strptime(tbase,'%H%M%S') - datetime.timedelta(seconds=4)).strftime('%H%M%S')
    vname1 = dbase + '_' + vtime1 + '.jpg'
    vname2 = dbase + '_' + vtime2 + '.jpg'

    if os.path.exists(('/').join(temp[:-1]) + '/' + vname1):
#         print(dfolder + '/' + vname1)
        os.rename(('/').join(temp[:-1]) + '/' + vname1, dfolder + '/' + vname1)
    if os.path.exists(('/').join(temp[:-1]) + '/' + vname2):
        os.rename(('/').join(temp[:-1]) + '/' + vname2, dfolder + '/' + vname2)


#     print('---Moved videos')
    return;


def findants2(vname):
    t = time.time()
#     print(vname,'asge')
    
    frames = loadendframes(vname, sep = 50)
    ndiff = subtractframes(frames)
    if ndiff<10500:
        deleteantvids(vname)
        print(vname, ' -- DELETED -- %.2f s' % (time.time()-t))
    else:
        movevids(vname)
        print(vname, ' -- MOVED -- %.2f s' % (time.time()-t))
    
    

In [12]:
# TEST LOOPING VS PARALLEL
# notes from ben: make a bash command that inputs file name and runs the python code you want, then put that in a for look.
# wait for ___ amount of time after every 16

# t = time.time()

# create array of how many frames ant present in
ant_present=np.zeros(shape=(int(len(vids)),1))

# pool = Pool(processes = 4)
# pool.map(findants, vids[197:204],3)

# result = pool.apply_async(findants, vids[197:198])

# for vid in vids[197:204]:

# #     result = pool.apply_async(findants, vids[197:198])
#     result = pool.apply_async(findants, [vid])
#     print(result.get(timeout=1))
#     time.sleep(5)
# #     print(result.get())


# for vid in vids[197:204]:
#     p = Process(target = findants, args = [vid])
#     p.start()


# for vname in vids[25:29]: #[129:130]: # total 131
#     findants(vname)
    
for vname in vids: #[129:130]: # total 131
    findants2(vname)
#     time.sleep(15)

        
# print('all done!!! -- %.2f s' % (time.time()-t))
print('all done!!! ')

/media/gravishlab/Seagate Expansion Drive/AntTrack/20180228_180639_16276718-0000.avi  -- MOVED -- 0.51 s
/media/gravishlab/Seagate Expansion Drive/AntTrack/20180228_180727_16276718-0000.avi  -- MOVED -- 0.73 s
/media/gravishlab/Seagate Expansion Drive/AntTrack/20180228_180814_16276718-0000.avi  -- MOVED -- 0.28 s
/media/gravishlab/Seagate Expansion Drive/AntTrack/20180228_180902_16276718-0000.avi  -- MOVED -- 0.77 s
/media/gravishlab/Seagate Expansion Drive/AntTrack/20180228_180959_16276718-0000.avi  -- MOVED -- 0.81 s
/media/gravishlab/Seagate Expansion Drive/AntTrack/20180228_181047_16276718-0000.avi  -- MOVED -- 0.81 s
/media/gravishlab/Seagate Expansion Drive/AntTrack/20180228_181209_16276718-0000.avi  -- MOVED -- 0.76 s
/media/gravishlab/Seagate Expansion Drive/AntTrack/20180228_181300_16276718-0000.avi  -- MOVED -- 0.76 s
/media/gravishlab/Seagate Expansion Drive/AntTrack/20180228_182526_16276718-0000.avi  -- MOVED -- 0.16 s
/media/gravishlab/Seagate Expansion Drive/AntTrack/2018

In [66]:
# TESTING WHY OPENCV KEEPS OLD FRAMES IN MEMORY

for vname in vids[14:15]:
    t = time.time()
    cap = cv2.VideoCapture(vname)
    vFr = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    print('---N_Frames', vFr)
    
    nFr = math.floor(vFr/10)+1
    frames = np.zeros((nFr, h, w), np.uint8)
    
    for kk,fr in enumerate(list(range(vFr))[::20]):
        cap.set(1,fr) # define frame (must be between 0 and 1)
#         tru, ret = cap.read(1)
        tru, ret = cap.read(fr)
    
        cv2.imshow('raw',ret)
        cv2.waitKey(10)
        
        if ((fr % 100) == 0):
            print('---',fr)
        
        # how close program
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
    
#     cap.release()
    cv2.destroyAllWindows()
#     del(ret)
#     del(cap)
#     gc.collect()
    print('all done!!! -- %.2f s' % (time.time()-t))

---N_Frames 717
--- 0
--- 100
--- 200
--- 300
--- 400
--- 500
--- 600
--- 700
all done!!! -- 9.69 s


In [25]:
# EXAMPLE FROM ONLINE

import multiprocessing
import time

def do_calculation(vname):
#     time.sleep(4)
    
    # capture video, and find number of frames
    cap = cv2.VideoCapture(vname)
    nFr = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    print('---N_Frames', nFr)
    
    frames = np.zeros((nFr, h, w), np.uint8)
    
    for kk,fr in enumerate(list(range(nFr))[::10]):
#         cap.set(1,fr) # define frame (must be between 0 and 1)
#         tru, ret = cap.read()
        tru, ret = cap.read(fr)

        # check if video frames are being loaded
        if not tru:
            print('Codec issue: cannot load frames.')
            break
            
        frames[kk,:,:] = ret[:,:,0]

        if ((fr % 100) == 0):
            print('---',fr)
            
    print('---Video loaded')
    cap.release()
    return frames

def start_process():
    print( 'Starting', multiprocessing.current_process().name)

if __name__ == '__main__':
    inputs = vids[0:2]
    print( 'Input   :', inputs)
    
    builtin_outputs = map(do_calculation, inputs)
    print ('Built-in:', builtin_outputs)
    
    pool_size = multiprocessing.cpu_count() * 2
    pool = multiprocessing.Pool(processes=pool_size,
                                initializer=start_process,
                                )
    pool_outputs = pool.map(do_calculation, inputs)
    pool.close() # no more tasks
    pool.join()  # wrap up current tasks

#     print( 'Pool    :', pool_outputs)
    print('this process done')

Input   : ['/media/gravishlab/Seagate Expansion Drive/AntTrack/20180220_140933_16276718-0000.avi', '/media/gravishlab/Seagate Expansion Drive/AntTrack/20180220_141149_16276718-0000.avi']
Built-in: <map object at 0x7fd6490f8b70>
Starting ForkPoolWorker-9
Starting ForkPoolWorker-10
Starting ForkPoolWorker-11
Starting ForkPoolWorker-12
Starting ForkPoolWorker-13
Starting ForkPoolWorker-14
Starting ForkPoolWorker-15
Starting ForkPoolWorker-16
Starting ForkPoolWorker-17
Starting ForkPoolWorker-18
Starting ForkPoolWorker-19
Starting ForkPoolWorker-20
Starting ForkPoolWorker-21
Starting ForkPoolWorker-22
Starting ForkPoolWorker-23
Starting ForkPoolWorker-24
---N_Frames 717
---N_Frames 717
--- 0
--- 0


Process ForkPoolWorker-22:
Process ForkPoolWorker-21:
Process ForkPoolWorker-19:
Process ForkPoolWorker-20:
Process ForkPoolWorker-24:
Process ForkPoolWorker-16:
Process ForkPoolWorker-23:
Process ForkPoolWorker-15:
Process ForkPoolWorker-18:
Process ForkPoolWorker-11:
Traceback (most recent call last):
Process ForkPoolWorker-14:
Process ForkPoolWorker-13:
Process ForkPoolWorker-12:
Process ForkPoolWorker-17:
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent c

KeyboardInterrupt: 

KeyboardInterrupt
KeyboardInterrupt
  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 108, in 

Starting ForkPoolWorker-25
Starting ForkPoolWorker-26
Starting ForkPoolWorker-27
Starting ForkPoolWorker-28
Starting ForkPoolWorker-29
Starting ForkPoolWorker-30
Starting ForkPoolWorker-31
Starting ForkPoolWorker-32
Starting ForkPoolWorker-33
Starting ForkPoolWorker-34
Starting ForkPoolWorker-35
Starting ForkPoolWorker-37
Starting ForkPoolWorker-38
Starting ForkPoolWorker-41


Process ForkPoolWorker-9:
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "<ipython-input-25-de84b23d345e>", line 21, in do_calculation
    tru, ret = cap.read(fr)
Process ForkPoolWorker-10:
KeyboardInterrupt
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File 

Starting ForkPoolWorker-43
Starting ForkPoolWorker-44


In [4]:
# EXAMPLE FROM ONLINE

# import multiprocessing

# def do_calculation(data):
#     return data * 2

# def start_process():
#     print( 'Starting', multiprocessing.current_process().name)

# if __name__ == '__main__':
#     inputs = list(range(197,204))
#     print( 'Input   :', inputs)
    
#     builtin_outputs = map(do_calculation, inputs)
#     print ('Built-in:', builtin_outputs)
    
#     pool_size = multiprocessing.cpu_count() * 2
#     pool = multiprocessing.Pool(processes=pool_size,
#                                 initializer=start_process,
#                                 )
#     pool_outputs = pool.map(do_calculation, inputs)
#     pool.close() # no more tasks
#     pool.join()  # wrap up current tasks

#     print( 'Pool    :', pool_outputs)

Input   : [197, 198, 199, 200, 201, 202, 203]
Built-in: <map object at 0x7fc2d07cf9e8>
Starting ForkPoolWorker-17
Starting ForkPoolWorker-18
Starting ForkPoolWorker-19
Starting ForkPoolWorker-20
Starting ForkPoolWorker-21
Starting ForkPoolWorker-22
Starting ForkPoolWorker-23
Starting ForkPoolWorker-24
Starting ForkPoolWorker-25
Starting ForkPoolWorker-26
Starting ForkPoolWorker-27
Starting ForkPoolWorker-28
Starting ForkPoolWorker-29
Starting ForkPoolWorker-30
Starting ForkPoolWorker-31
Starting ForkPoolWorker-32
Pool    : [394, 396, 398, 400, 402, 404, 406]


In [6]:
# %qtconsole

In [4]:
# # find list of all videos
# vids = glob.glob('/media/gravishlab/Seagate Expansion Drive/AntTrack/Rugosity_1_41/*18-0000.avi')
# print('Total Number of Videos: ',len(vids))

Total Number of Videos:  0


In [ ]:
# FOR LOOP METHOD

# for vname in vids[197:198]:

#     cap = cv2.VideoCapture(vname)
#     nFr = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Set-up background subtraction
#     fgbg = cv2.createBackgroundSubtractorMOG2()

#     total = []
#     frames = []
    
#     for Fr in list(range(0,716))[0::10]:
#         print('Fr: ', Fr)
        
#         # read in raw frame and get dimensions
#         cap.set(1,Fr) # define frame (must be between 0 and 1)
#         ret, frame = cap.read()
#         if not ret:
# #             print('---End of video!')
#             break
#         [h,w,ch] = frame.shape
        
        # creat blank image to show versions next to eachother
#         montage = np.zeros((h,w*4,1),np.uint8)
    
        # apply background subtraction
#         fgmask = fgbg.apply(frame)
        
        # get rid of salt and pepper noise
#         median_blur= cv2.medianBlur(fgmask,3)
#         median_blur= cv2.medianBlur(bkgd_div[Fr,:,:],3)
        
        # find how many points are above a threshold
#         thresholded=median_blur[0+50-1:(h-50-1), 0+50-1:(w-120-1)]<.8 # only check within a ROI
#         total[Fr-1,0]=thresholded.sum() # total number of pixels above threshold
#         total = np.append(total, thresholded.sum())
    
    
        # PLOT THINGS
        
#         # draw ROI cutoff on images
#         cv2.rectangle(median_blur, (0+50-1,0+50-1), (w-120-1,h-50-1), (255,255,0), thickness=2, lineType=8, shift=0)
#         cv2.rectangle(frame, (0+50-1,0+50-1), (w-120-1,h-50-1), (255,255,0), thickness=2, lineType=8, shift=0)
        
# #         # show in separate windows
#         cv2.imshow('raw',frame)
#         cv2.imshow('background subtract',fgmask)
#         cv2.imshow('background subtract',bkgd_div[Fr,:,:])
# # #         cv2.imshow('median blur',median_blur)
        
        # put into montage, resize, and display
#         montage[:,0:w]=frame[:,:,1:2]
#         montage[:,w:2*w]=np.reshape(fgmask,(h,w,1)) 
# #         montage[:,2*w:3*w]=np.reshape(median_blur,(h,w,1)) 
#         cv2.putText(montage,str(Fr),(30,50), cv2.FONT_HERSHEY_PLAIN, 2, (255,255,255),2, cv2.LINE_AA)
# #         cv2.putText(montage,str(thresholded.sum()),(2*w+30,50), cv2.FONT_HERSHEY_PLAIN, 2, (255,255,255),2, cv2.LINE_AA)
#         cv2.imshow('image',cv2.resize(montage,(0,0),fx=0.5,fy=0.5))
#         cv2.waitKey(50)


#     if ant_frames.sum() < 4:
#         os.remove(vname)
#         if os.path.exists(vname[0:len(vname)-11] + '12-0000.avi'):
#             os.remove(vname[0:len(vname)-11] + '12-0000.avi')
#         if os.path.exists(vname[0:len(vname)-11] + '35-0000.avi'):
#             os.remove(vname[0:len(vname)-11] + '35-0000.avi')
#         if os.path.exists(vname[0:len(vname)-11] + '36-0000.avi'):
#             os.remove(vname[0:len(vname)-11] + '36-0000.avi')


#     else: # MOVE TO NEW LOCATION
#         os.rename(vname, dfolder + vname[len(vname)-34:len(vname)-11] + '12-0000.avi')
#         if os.path.exists(vname[0:len(vname)-11] + '18-0000.avi'):
#             os.rename(vname[0:len(vname)-11] + '18-0000.avi', dfolder + vname[len(vname)-34:len(vname)-11] + '18-0000.avi')
#         if os.path.exists(vname[0:len(vname)-11] + '35-0000.avi'):
#             os.rename(vname[0:len(vname)-11] + '35-0000.avi', dfolder + vname[len(vname)-34:len(vname)-11] + '35-0000.avi')
#         if os.path.exists(vname[0:len(vname)-11] + '36-0000.avi'):
#             os.rename(vname[0:len(vname)-11] + '36-0000.avi', dfolder + vname[len(vname)-34:len(vname)-11] + '36-0000.avi')
#         print('---Videos moved')


    # When everything done, release the capture
#     cap.release()
#     cv2.destroyAllWindows()

In [5]:
# find list of all videos
print(dfolder)
vids = glob.glob(dfolder + '/*18-0000.avi')
print('Total Number of Videos: ',len(vids))

/media/gravishlab/Seagate Expansion Drive/AntTrack
Total Number of Videos:  413
